In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.3.0'

In [3]:
print "Author: Tommy Mulc"

Author: Tommy Mulc


Create a TensorFlow cluster with one node.  Let this node be responsible for a job that that has name "worker" and that will operate one take at localhost:2222

In [4]:
cluster_spec = tf.train.ClusterSpec({'worker' : ['localhost:2222'], 'ps' : ['localhost:2223']})
server = tf.train.Server(cluster_spec,job_name='worker')

# Create variables on graph

In [6]:
# sess = tf.Session(target=server.target)

In [7]:
#tf.reset_default_graph()

In [8]:
# a = tf.Variable([0.],name='a')
# b = tf.constant([100.])
# loss = tf.abs(a-b)

In [9]:
# tf.global_variables()

[<tf.Variable 'a:0' shape=(1,) dtype=float32_ref>]

In [10]:
tf.local_variables()

[]

In [11]:
for v in tf.global_variables():
    tf.add_to_collection('local_variables',v)

In [12]:
tf.local_variables()

[<tf.Variable 'a:0' shape=(1,) dtype=float32_ref>]

One variable appears in the trainable collection.

In [13]:
tf.get_collection('trainable_variables')

[<tf.Variable 'a:0' shape=(1,) dtype=float32_ref>]

In [14]:
init = tf.global_variables_initializer()
init_local = tf.local_variables_initializer()

create local references

In [15]:
a_local = [v  for v in tf.local_variables() if v.name == 'a:0'][0]

In [16]:
sess = tf.Session(target=server.target)

In [19]:
#sess.run(a) #will give uninitialized error
#sess.run(a_local) #will give uninitialized error

In [20]:
sess.run(init_local)

In [21]:
# works after only doing local init, because the variableis both a local and global variables!
sess.run(a)

array([ 0.], dtype=float32)

# Local and global variables on same graph

### Initialization

In [5]:
tf.reset_default_graph()

In [6]:
local_variable = tf.Variable([0.],name='local',collections=[tf.GraphKeys.LOCAL_VARIABLES])
global_variable = tf.Variable([0.],name='global',collections=[tf.GraphKeys.GLOBAL_VARIABLES])
both_variable = tf.Variable([0.],name='both',
                            collections=[tf.GraphKeys.GLOBAL_VARIABLES,tf.GraphKeys.LOCAL_VARIABLES])
local_init = tf.local_variables_initializer()
global_init = tf.global_variables_initializer()

In [7]:
sess = tf.Session()
#sess = tf.Session(target=server.target,graph=g)

In [8]:
sess.run(local_init)

In [9]:
print("Local: ",sess.run(local_variable))
print("Both: ",sess.run(both_variable))
#print("Global: ",sess.run(global_variable)) # won't run

('Local: ', array([ 0.], dtype=float32))
('Both: ', array([ 0.], dtype=float32))


In [10]:
sess.run(global_init)

In [11]:
print("Global: ",sess.run(global_variable))

('Global: ', array([ 0.], dtype=float32))


**Even if you reset the graph, variables don't need to be reinitialized. This only occurs when the session target is the same server (and if the same server has not been reset.**

## Device mapping

With the above graph, look at the device mapping

In [12]:
print(local_variable.device)
print(global_variable.device)
print(both_variable.device)

Create a new graph with variables on parameter servers and other ops on the worker device

In [13]:
tf.reset_default_graph()

In [14]:
with tf.device('/job:ps/task:0'):
    a = tf.Variable([0.],name='a')
with tf.device('/job:worker/task:0'):
    b = tf.constant([100.])
    loss = tf.abs(a-b)
    init = tf.global_variables_initializer()

In [15]:
print(a.device)
print(b.device)
print(loss.device)
print(init.device)

/job:ps/task:0
/job:worker/task:0
/job:worker/task:0
/job:ps/task:0


Note that the init op got placed on the ps althought we explicitly stated not to...  But if you explicityly put the init op on the ps, then the op will never run when called from a worker.

In [16]:
sess = tf.Session(target=server.target)
sess.run(init)
sess.run([loss])

[array([ 100.], dtype=float32)]

If you forgot to launch the ps before running the above cell, the notebook will just stall until the ps is launched and then it will execute

Put some variables one ps some not

In [10]:
tf.reset_default_graph()

In [11]:
with tf.device('/job:ps/task:0'):
    a = tf.Variable([0.],name='a')
with tf.device('/job:worker/task:0'):
    b = tf.constant([100.])
    c = tf.Variable([0.],name='a')
    loss = tf.abs(a-b)
    #init = tf.global_variables_initializer()
with tf.device('/job:ps/task:0'):
    init_ps = tf.global_variables_initializer()

In [12]:
print(a.device)
print(b.device)
print(loss.device)
#print(init.device)
print(init_ps.device)

/job:ps/task:0
/job:worker/task:0
/job:worker/task:0
/job:ps/task:0


In [ ]:
sess = tf.Session(target=server.target)
sess.run(init_ps)

In [ ]:
sess.run([c])

# Parameter Server

Goal will  be to store the variables

In [203]:
# DO NOT RUN IN THIS NOTEBOOK
# Will continue to join until Jupyter is killed
cluster_spec = tf.train.ClusterSpec({'ps' : ['localhost:2223'],'worker' : ['localhost:2224']})
ps = tf.train.Server(cluster_spec,job_name='ps')
ps.join()

# Using device setter

In [13]:
tf.reset_default_graph()

In [15]:
cluster_spec = tf.train.ClusterSpec({'ps' : ['localhost:2223'],'worker' : ['localhost:2224']})
#server = tf.train.Server(cluster_spec,job_name='worker')

In [16]:
with tf.device(tf.train.replica_device_setter(ps_tasks=1,cluster=cluster_spec)):
    a = tf.Variable([0.],name='a')
    b = tf.constant([100.])
    loss = tf.abs(a-b)
    
    opt = tf.train.GradientDescentOptimizer(.1).minimize(loss)
#     with tf.device('/job:worker/task:0'):
#         for v in tf.global_variables():
#             tf.add_to_collection('local_variables',v)

In [17]:
print(a.device)

/job:ps/task:0


Create local copies

In [20]:
for v in tf.trainable_variables():
    print v.device

a


In [23]:
with tf.device('/job:worker/task:0'):
    local_anchor = tf.no_op()
    with tf.colocate_with(local_anchor):
        for v in tf.trainable_variables() :
            v_l = tf.get_variable(name='local/%s'%(v.op.name),
                initializer=v.initialized_value(),
                trainable=False,
                collections=[tf.GraphKeys.LOCAL_VARIABLES])

In [26]:
for v in tf.local_variables():
    print v.device

/job:worker/task:0


In [108]:
with tf.device(tf.train.replica_device_setter(ps_tasks=1,cluster=cluster_spec)):
    with tf.device('/job:worker/task:0'), tf.name_scope(""):
        for v in tf.global_variables():
            tf.add_to_collection('local_variables',v)
        a_local = [v  for v in tf.local_variables() if v.name == 'a:0'][0]
        b = tf.Variable(1.,name='b')
        tf.get_default_graph().clear_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        
        local_anchor = tf.no_op()
        with tf.colocate_with(local_anchor):
            for v in tf.trainable_variables() :
                v_g = tf.get_variable(name='%s/g/%s'%(tf.get_variable_scope().name, v.op.name),
                    initializer=v.initialized_value(),
                    trainable=False,
                    collections=[tf.GraphKeys.GLOBAL_VARIABLES])
#         v_g = variable_scope.get_variable(
#             name='%s/g/%s' % (self._name, v.op.name),
#             initializer=v.initialized_value(),
#             trainable=False,
#             collections=[ops.GraphKeys.GLOBAL_VARIABLES, 'global_model'])

ValueError: '/g/a' is not a valid scope name

In [107]:
tf.get_variable_scope().name

''

In [95]:
fuck = tf.get_variable_scope()
fuck.get_variable()

In [80]:
print(a_local.device,b.device)

(u'/job:ps/task:0', u'/job:worker/task:0')


In [50]:
sess = tf.Session(target=server.target)

In [51]:
# with tf.device('/job:worker/task:0'):
#     for v in tf.global_variables():
#         tf.add_to_collection('local_variables',v)
#     a_local = [v  for v in tf.local_variables() if v.name == 'a:0'][0]